In [63]:
# Importing required items
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
import pandas as pd
import shutil
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [64]:
# Confirm the connection to the GPU with tensorflow
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [65]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jul 11 19:58:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    39W / 300W |    596MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [66]:
# Determine the available RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [93]:
# Defining the training and testing directories
training_data_dir = '/content/drive/MyDrive/images/train'
test_data_dir = '/content/drive/MyDrive/images/test'



In [68]:
# Defining hyperparameters
img_width, img_height = 150, 150
batch_size = 15
epochs = 30
num_classes = 12  # Number of different types in dataset

In [69]:
# Defining data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [70]:
test_datagen = ImageDataGenerator(rescale=1/255)

In [71]:
# Defining training generator
train_generator = train_datagen.flow_from_directory(
    training_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4750 images belonging to 12 classes.


In [72]:
# Defining the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [73]:
# Adjusting learning rate and change optimization algorithm
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

In [74]:
# Compiling the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

In [75]:
# Optimizing and improving memory usage
tf.config.optimizer.set_jit(True)

In [76]:
# Training the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
)

Epoch 1/30
316/316 [==============================] - 2223s 7s/step - loss: 1.8913 - accuracy: 0.3582
Epoch 2/30
316/316 [==============================] - 98s 309ms/step - loss: 1.3392 - accuracy: 0.5385
Epoch 3/30
316/316 [==============================] - 92s 293ms/step - loss: 1.1170 - accuracy: 0.6207
Epoch 4/30
316/316 [==============================] - 97s 307ms/step - loss: 0.9962 - accuracy: 0.6617
Epoch 5/30
316/316 [==============================] - 92s 293ms/step - loss: 0.9140 - accuracy: 0.6921
Epoch 6/30
316/316 [==============================] - 94s 297ms/step - loss: 0.8391 - accuracy: 0.7231
Epoch 7/30
316/316 [==============================] - 92s 291ms/step - loss: 0.7714 - accuracy: 0.7373
Epoch 8/30
316/316 [==============================] - 95s 301ms/step - loss: 0.7261 - accuracy: 0.7567
Epoch 9/30
316/316 [==============================] - 92s 289ms/step - loss: 0.7080 - accuracy: 0.7565
Epoch 10/30
316/316 [==============================] - 92s 292ms/step - lo

In [77]:
# Saving the trained model
model.save('project4_model.h5')

In [104]:
# Loading the trained model
model = tf.keras.models.load_model('project4_model.h5')

In [105]:
# Creating an ImageDataGenerator for the testing dataset
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [106]:
# Loading the filenames of the images in the testing dataset
test_filenames = os.listdir(test_data_dir)

In [107]:
# Creating a dataframe with the filenames
test_df = pd.DataFrame({'filename': test_filenames})
print(test_filenames)
print(len(test_filenames))

['e98e5d1d5.png', 'fea1d13d6.png', 'fda0b5c38.png', 'fdea6b119.png', 'fe29629fb.png', 'fd925f542.png', 'fda39e16f.png', 'fe9e87b78.png', 'fd253a74e.png', 'fd87b36ae.png', 'fc6f686fb.png', 'fc3e58836.png', 'e5368474f.png', 'dabe3e5be.png', 'cfd8165e9.png', 'cfb18d262.png', 'cf90fc52d.png', 'cf46d09c5.png', 'e1abb4ff9.png', 'caa2fbd79.png', 'efe19dc32.png', 'cf3a8b2fd.png', 'f4caf74f9.png', 'f1f7c833f.png', 'f3fcfff1b.png', 'f2dc546ca.png', 'f1e87cba7.png', 'ffc6f8527.png', 'f4ad9d950.png', 'f4e7733d4.png', 'f445fe6fb.png', 'f351ce097.png', 'fbf88b6be.png', 'ff65bc002.png', 'f66ae4070.png', 'f85ed9b6d.png', 'fef2ade8c.png', 'fba8fc78a.png', 'fb022edf9.png', 'fa9f3a8f9.png', 'fa5fd1384.png', 'fadc6adbc.png', 'f48916a8c.png', 'f25996db8.png', 'f4234cf4f.png', 'f8318faf1.png', 'f4021df6c.png', 'f593c9cf0.png', 'f33e9d918.png', 'f23faf9c1.png', 'f9b6bfb00.png', 'f9ea23fb5.png', 'f6d250856.png', 'f9f35cbd4.png', 'c0f5d9ac8.png', 'fea355851.png', 'fea3da57c.png', 'b7192c70f.png', 'd7017f701.pn

In [108]:
# Generating the test data generator from the dataframe
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=test_data_dir,
    x_col='filename',
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

Found 794 validated image filenames.


In [109]:
# Predicting the classes for the test images
predictions = model.predict(test_generator)

53/53 [==============================] - 7s 103ms/step


In [110]:
# Maping predicted class indices to class labels
class_labels = list(train_generator.class_indices.keys())
predicted_classes = [class_labels[prediction.argmax()] for prediction in predictions]

In [111]:
# Printing the predicted classes
print(predicted_classes)

['Common wheat', 'Small-flowered Cranesbill', 'Common wheat', 'Black-grass', 'Loose Silky-bent', 'Common Chickweed', 'Loose Silky-bent', 'Scentless Mayweed', 'Cleavers', 'Loose Silky-bent', 'Sugar beet', 'Scentless Mayweed', 'Scentless Mayweed', 'Small-flowered Cranesbill', 'Charlock', 'Scentless Mayweed', 'Maize', 'Common Chickweed', 'Sugar beet', 'Common Chickweed', 'Charlock', 'Loose Silky-bent', 'Charlock', 'Shepherds Purse', 'Charlock', 'Cleavers', 'Loose Silky-bent', 'Common wheat', 'Sugar beet', 'Small-flowered Cranesbill', 'Charlock', 'Loose Silky-bent', 'Charlock', 'Charlock', 'Maize', 'Shepherds Purse', 'Sugar beet', 'Black-grass', 'Fat Hen', 'Cleavers', 'Shepherds Purse', 'Charlock', 'Scentless Mayweed', 'Small-flowered Cranesbill', 'Fat Hen', 'Common Chickweed', 'Scentless Mayweed', 'Sugar beet', 'Scentless Mayweed', 'Loose Silky-bent', 'Scentless Mayweed', 'Fat Hen', 'Shepherds Purse', 'Common Chickweed', 'Small-flowered Cranesbill', 'Loose Silky-bent', 'Sugar beet', 'Comm

In [112]:
# Adding the predicted classes to the dataframe
test_df['predictions'] = predicted_classes
test_df.head()

,filename,predictions
0,e98e5d1d5.png,Common wheat
1,fea1d13d6.png,Small-flowered Cranesbill
2,fda0b5c38.png,Common wheat
3,fdea6b119.png,Black-grass
4,fe29629fb.png,Loose Silky-bent


In [113]:
# Creating the predicted subfolders for each type
predicted_dir = '/content/drive/MyDrive/images/predictions'
os.makedirs(predicted_dir, exist_ok=True)
for class_label in class_labels:
    class_dir = os.path.join(predicted_dir, class_label)
    os.makedirs(class_dir, exist_ok=True)

In [114]:
# Copying the images to their predicted folders
for filename, predicted_class in zip(test_filenames, predicted_classes):
    source_path = os.path.join(test_data_dir, filename)
    destination_path = os.path.join(predicted_dir, predicted_class, filename)
    shutil.copy(source_path, destination_path)

print("Images have been copyed to their respective predicted folders.")

Images have been copyed to their respective predicted folders.
